# Stage 2. LDA Topic modeling

Topic modeling was perfomed on the science fiction corpus chunking the novels at the page level using the scikit-learn Latent Dirichlet allocation (LDA) library.

Three were the algorithm's parameters I decided to manipulate in order to fine-tune my model. Max_df, which regulates the presence of highly occurring words in the word-document matrix, was set either to 0.25 or 0.30. Hence, only the words that occurred in less than 25% or 30% of the documents were considered. Max_features, which regulates the number of unique words and the presence of rare words in the matrix, was set to 1000 or 10000. I decided to keep 10000 in order to have less common words in the model too and so that the presence of highly occurring once would not sway the model as much. N_components, which regulates the number of topics in the model, was set to 50, 60, 70, 80, 90, 100, 120. This is the parameter that most affects the results and there. Several attempts were made by changing these parameters.

This code was written and provided by Matthew Wilkens. I will signal only the few lines I have written at the end of the notebook to store the results of the algorithm.


## 1. Imports, variables and downloads

In [0]:
%matplotlib inline

import pandas as pd
import os
import sys
import numpy as np
import glob
import gzip

from   htrc_features import FeatureReader, utils as frutils
from   nltk.stem import WordNetLemmatizer
from   sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from   sklearn.decomposition import LatentDirichletAllocation


# Directories for input and output
figDir = 'figures'
resultsDir = 'results'
inputDir = 'inputs'

# Full corpus data can be large; make it easy to stash outside GitHub/Google
bigDir = '.' # Base directory for large files
htrcefDir = os.path.join(bigDir, 'htrcef') # HTRC-EF JSONs
corpus_file = os.path.join(bigDir, 'corpus.txt.gz') # Text version of corpus
corpus_ids_file = os.path.join(bigDir, 'corpus_ids.txt.gz') # Corpus identifiers
corpus_file_trim = os.path.join(bigDir, 'corpus_trimmed.txt.gz') # Trimmed version of corpus
corpus_ids_file_trim = os.path.join(bigDir, 'corpus_trimmed_ids.txt.gz') # Trimmed corpus ids
logfile = os.path.join(bigDir, 'lda.log') # Log file (for Gensim)

os.makedirs(figDir, exist_ok=True)
os.makedirs(resultsDir, exist_ok=True)
os.makedirs(inputDir, exist_ok=True)
os.makedirs(htrcefDir, exist_ok=True)

# Variables that affect processing
reprocess = True # Discard very short pages?
stoplist_file = os.path.join(inputDir, 'stopwords-underwood-goldstone.txt')

###Science fiction HTIDs


In [0]:
# List of HTIDs to use
import csv

def creating_volid_list(csv_file):
    
    list_htids = []
    
    with open(csv_file, 'r', encoding='utf-8') as open_csv:
        dict_csv = csv.DictReader(open_csv)
        for row in dict_csv:
            htid = str(row["htid"])
            if "None" not in htid and htid != "":
                list_htids.append(htid)

        return list_htids
    
volids = creating_volid_list("metadata_with_htids.csv")

print(len(volids))

331


### Download HTRC-EF files

In [0]:
# Download the extracted features files for all volumes in the corpus
frutils.download_file(htids=volids, outdir=htrcefDir)

(0, None)

## 2. Process corpus

This part of the code is meant to process the text corpus in order to stream it through the algorithm. The novels are chunked by page. Two gzipped flat text files are created: the first contains the htid of the novel and the page number; in the second each page's words are written as one line strings.

### Cleaning and turning corpus to lists of string

Short pages and the few pages at the beginning and end of each novel are discarded as they are likely paratext. Only the tokens tagged as the selected parts of speech are retained. Stopwords are removed. The remaining words are lemmatized.

In [0]:
# Turn EF volumes into one-page-per-line corpus
# Remove stopwords, select parts of speech, lemmatize, and lowercase

# Penn treebank tags to keep
pos_to_include = [
    'FW',  # foreign
    'JJ',  # adjectives
    'JJR',
    'JJS',
    'MD',  # modal
    'NN',  # nouns (not proper)
    'NNS',
    'RB',  # adverbs
    'RBR',
    'RBS',
    'VB',  # verbs
    'VBD',
    'VBG',
    'VBN',
    'VBP',
    'VBZ'
]


# Functions to work with EF volumes
def encode_volid(volid, direction='path'):
    '''
    Transform htid into filename encoded version and vice versa
    '''
    encoding_fixes = {'+':':', '=':'/'}
    if direction=='path':
        encoding_fixes = {v:k for k,v in encoding_fixes.items()}
    for key in encoding_fixes:
        volid = volid.replace(key, encoding_fixes[key])
    return(volid)


# Translate Penn->WordNet PoS tags
#  Need WordNet PoS tags for lemmatizer
def get_wordnet_pos(treebank_tag):
    from nltk.corpus import wordnet
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('M'):
        return wordnet.VERB
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Transform EF page to space-delimited string
def efpage2doc(page, cutoff=50):
    doc = ''
    if page.token_count() >= cutoff:
        tokens = page.tokenlist(case=False).query('pos in @pos_to_include')
        for token in tokens.itertuples():
            word = token.Index[2]
            if word not in stoplist:
                pos = get_wordnet_pos(token.Index[3])
                word = lemmatizer.lemmatize(word, pos=pos)
                for i in range(token.count):
                    doc += word+' '
    return(doc.strip())

# Transform EF volume into list of one-line page-level document strings
def efvol2docs(vol, skip_first=10, skip_last=10, min_page_tokens=50):
    docs = []
    pages = []
    for page in vol:
        if (skip_first < int(page.seq) < (vol.page_count-skip_last)):
            docs.append(efpage2doc(page, min_page_tokens))
            pages.append(page.seq)
    return(pages, docs)

In [0]:
# Read stoplist and set up
min_page_tokens = 50 # Ignore pages with fewer than this many tokens
skip_first_pages = 10 # Skip first and last pages of each book (paratext)
skip_last_pages = 10

stoplist = [line.strip() for line in open(stoplist_file)]
stoplist = set(stoplist)
print("Words in stoplist:", len(stoplist))

Words in stoplist: 6048


### Perform first processing pass

The two algorithms above are part of the following greater function that creates two gzipped text files. In `corpus_file` each line is a page of a novel. Whereas, in `corpus_ids_file`, each line is the htid and page number of a novel. As result, it is possible to later reconstruct which pages belong to which books in what order.


In [0]:
%%time
# Perform corpus processing
lemmatizer = WordNetLemmatizer() # Initialize lemmatizer
vols_processed = 0 # Count volumes processed
vols_error = [] # Keep track of volumes with errors

# Read HTRC-EF files, process, write out as single (gzipped) corpus file
with gzip.open(corpus_ids_file, 'wt') as fi:
    with gzip.open(corpus_file, 'wt', encoding='utf-8') as fd:
        for volid in volids:
            try:
                vol = FeatureReader(
                    os.path.join(
                        htrcefDir,
                        f'{encode_volid(volid)}.json.bz2'
                    )).first()
                pages, docs = efvol2docs(
                    vol, 
                    skip_first=skip_first_pages,
                    skip_last=skip_last_pages,
                    min_page_tokens=min_page_tokens
                )
                for doc in docs:
                    fd.write(doc+'\n')
                for page in pages:
                    fi.write(f'{volid} {page}\n')
            except:
                vols_error.append(volid)
            vols_processed += 1

print(f'Processed {vols_processed} vols with {len(vols_error)} errors')

Processed 331 vols with 0 errors
CPU times: user 35min 15s, sys: 2.42 s, total: 35min 17s
Wall time: 35min 18s


### Perform second processing pass

It's possible for the above corpus setup to produce some page-level documents that are empty or very short. This may cause errors. We can reprocess to remove those pages.

In [0]:
corpus_to_use = corpus_file
ids_file = corpus_ids_file

if reprocess:
    corpus_to_use = corpus_file_trim
    ids_file = corpus_ids_file_trim

    # Reprocess corpus to remove empty and very short docs
    from collections import deque
    indices_to_delete = deque()
    counter = 0
    with gzip.open(
        corpus_file_trim,
        'wt', 
        encoding='utf-8'
    ) as f:    
        for line in gzip.open(corpus_file, 'rt', encoding='utf-8'):
            if len(line.strip().split()) < 10:
                indices_to_delete.append(counter)
            else:
                f.write(line)
            counter += 1
    to_delete = indices_to_delete.copy()
    skip = to_delete.popleft()
    counter = 0
    with gzip.open(
        corpus_ids_file_trim,
        'wt',
        encoding='utf-8'
    ) as f:
        for line in gzip.open(corpus_ids_file, 'rt', encoding='utf-8'):
            if counter == skip:
                if len(to_delete) > 0:
                    skip = to_delete.popleft()
                else:
                    pass
            else:
                f.write(line)
            counter += 1

## 3. Core of the code

### Vectorize corpus

Turn texts into a document-term matrix with specified parameters.

In [0]:
# Streaming corpus, precomputed length
#  Use a streaming version of the corpus for memory efficiency
class StreamCorpus(object):
    def __init__(self, obs=None):
        i = 0
        for line in gzip.open(corpus_to_use, 'rt', encoding='utf-8'):
            i+=1
        self._data_len = i
    def __iter__(self):
        for line in gzip.open(corpus_to_use, 'rt', encoding='utf-8'):
            yield (line)
    def __len__(self):
        return self._data_len
    

corpus = StreamCorpus()
ids = [line.strip() for line in gzip.open(ids_file, 'rt')]

# LDA settings
max_df = 0.30         # Keep words that appear on no more than x fraction of pages
                      # This removes high-frequency words not already included on
                      # stopwords list.

min_df = 3            # Keep words that appear on at least x total pages
                      # Removes very rare words
                      # Use larger value for larger corpora

max_iter = 20         # Number of LDA passes over corpus

n_features = 10000    # Max unique words to use in model
                      # Another way to remove very low-frequency words
                      # Use larger value for larger corpora

n_components = 80     # Number of topics
                      # Use larger value for larger corpora

# Regular vectorizer. Not memory-efficient, but easy to use
count_vectorizer = CountVectorizer(
    encoding='utf-8',
    max_df=max_df, 
    min_df=min_df,
    max_features=n_features
)

# Pick a vectorizer
vectorizer = count_vectorizer

# Perform vectorization
tf = vectorizer.fit_transform(corpus)
print(tf.shape)

(93286, 10000)


### Perform LDA

In [0]:
%%time
%env JOBLIB_TEMP_FOLDER=/tmp

lda = LatentDirichletAllocation(
    n_components=n_components, 
    max_iter=max_iter,
    learning_method='online',
    learning_offset=50.,
    random_state=0,
    n_jobs=1
)

env: JOBLIB_TEMP_FOLDER=/tmp
CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 6.63 ms


In [0]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=20, mean_change_tol=0.001,
             n_components=80, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

## 4. Preliminary overview of the results

### Topic keywords

In [0]:
n_top_words = 10     # Print this many words per topic
def print_top_words(model, feature_names, n_top_words):
    '''
    Print the top words associated with each modeled topic.
    '''
    index_words = []
    print("\nTopics in LDA model:\n==========")
    for topic_idx, topic in enumerate(model.components_): #num of the topic and the topic itself
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] #get word/feature from the vectorizer array
                             for i in topic.argsort()[:-n_top_words - 1:-1]])  #select ten last indeces that correspond to top words in the topic
        print(message)
        index_words.append(message.split(' ')[2])
    print()
    return index_words

#each topic in the model contains all the words/features in the same order as the array of words/features in the vectorizer

tf_feature_names = vectorizer.get_feature_names()
topic_index_words = print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: cell dragon monster beard suck jungle net smell dwarf beast
Topic #1: all act resume propose hostile aura suppress confuse determination trifle
Topic #2: black white wear fish red dress brown shirt blue hair
Topic #3: body spine jaw teeth mouth flesh bone gill corpse wan
Topic #4: technician interstellar stallion designer peach rating earthling motorcycle orgasm coastal
Topic #5: battle mad pretend chill console stable mask behave fighter skinny
Topic #6: shake head sit room smile chair table face nod turn
Topic #7: re ll ve want right let good maybe try ask
Topic #8: boat suit sand station pilot engine bridge drive launch cabin
Topic #9: read book write letter page story thumb chapter novel author
Topic #10: commander circuit an transmission sport monk electronic calculate reflex uneasily
Topic #11: run foot away head arm side pull again leg ground
Topic #12: love never child woman life give mother own world heart
Topic #13: water river north wind land 

### Scoring

Not doing anything with this now, but could help determine the "best" number of topics by creating multiple models and optimizing `score()` and `perplexity()` outputs. Values are meaningful only in comparison to other models of the same data. There is no abstractly "good" score.

In [0]:
print("Log-likelihood score:", round(lda.score(tf), 2))
print("Perplexity:", round(lda.perplexity(tf), 2))

Log-likelihood score: -96275597.05
Perplexity: 6493.67


### Doc-topic matrix

Create a pandas dataframe containing topic fractions for each document. Each "document" is one page of a volume for our purposes.

In [0]:
# Transform input vectors to topics
lda_output = lda.transform(tf)

# column names
topicnames = [f"t{str(i)} {topic_index_words[i]}" for i in range(lda.n_components)]

# index names
docnames = ids

# Make the dataframe
dtm = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(dtm.values, axis=1)
dtm['dominant_topic'] = dominant_topic

# Add htid and page number for each document
htids = []
pagenos = []
for docid in dtm.index:
    htid, pageno = docid.split()
    htids.append(htid)
    pagenos.append(pageno)
dtm['htid'] = htids
dtm['page'] = pagenos

print("Sample rows of the document-topic matrix:")
display(dtm.sample(10))

Sample rows of the document-topic matrix:


,t0 cell,t1 all,t2 black,t3 body,t4 technician,t5 battle,t6 shake,t7 re,t8 boat,t9 read,...,t73 war,t74 death,t75 virus,t76 require,t77 drink,t78 clan,t79 mile,dominant_topic,htid,page
mdp.39015020711092 00000296,0.01,0.0,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.03,0.00,0.02,0.06,0.0,0.04,43,mdp.39015020711092,00000296
mdp.49015002042985 00000127,0.06,0.0,0.00,0.00,0.00,0.00,0.00,0.11,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.0,0.00,11,mdp.49015002042985,00000127
uc1.32106002188628 00000300,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.08,0.00,0.00,...,0.0,0.23,0.00,0.00,0.00,0.0,0.00,43,uc1.32106002188628,00000300
mdp.39015021474286 00000223,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.04,0.01,0.00,0.00,0.0,0.06,43,mdp.39015021474286,00000223
mdp.39015005343515 00000218,0.02,0.0,0.00,0.00,0.00,0.00,0.05,0.22,0.01,0.00,...,0.0,0.06,0.00,0.00,0.01,0.0,0.00,7,mdp.39015005343515,00000218
mdp.39015034647167 00000091,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.0,0.00,28,mdp.39015034647167,00000091
inu.30000082136767 00000056,0.00,0.0,0.16,0.00,0.00,0.00,0.00,0.00,0.00,0.06,...,0.0,0.00,0.00,0.00,0.01,0.0,0.00,48,inu.30000082136767,00000056
mdp.39015017656870 00000103,0.00,0.0,0.00,0.00,0.01,0.00,0.00,0.03,0.00,0.00,...,0.0,0.15,0.00,0.00,0.00,0.0,0.00,48,mdp.39015017656870,00000103
mdp.39015034512692 00000105,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.03,0.00,...,0.0,0.04,0.00,0.01,0.00,0.0,0.00,43,mdp.39015034512692,00000105
mdp.39015082679633 00000041,0.00,0.0,0.00,0.01,0.00,0.00,0.00,0.05,0.04,0.00,...,0.0,0.12,0.00,0.01,0.00,0.0,0.00,60,mdp.39015082679633,00000041


In [0]:
# Most frequently dominant topics
dtm.dominant_topic.value_counts()

## Storing the results

### Visualization

Compute, display and save a handy HTML visualization of the topic model output.

In [0]:
%%time
import pyLDAvis
import pyLDAvis.sklearn
visdata = pyLDAvis.sklearn.prepare(lda, tf, vectorizer)
pyLDAvis.save_html(visdata, os.path.join(resultsDir, 'pyLDAvis-80_10000_0.30.html'))

CPU times: user 1min 18s, sys: 1.68 s, total: 1min 20s
Wall time: 1min 27s


### Save doc-topic matrix to CSV file

In [0]:
#Grouping records of the dataframe by HTID to have the percentages of each topic for each novel
#And save to a CSV file
dtm.groupby('htid').mean().to_csv(os.path.join(resultsDir, 'lda_80_10000_0.30_htid.csv'))

## 5. Highest scoring books for city topic

Retrieval of the metadata regarding the highest scoring books for estimated presence of city topic. I identified topic 19 in the html visualization to be city topic since it is the most interested in city space.

Code by me

In [0]:
citytopic_htids = pd.read_csv("/content/drive/My Drive/Università/3 ANNO MAGISTRALE/TESI/3_topicmodeling/lda_80_10000_0.30_htid.csv")[["htid", "t36 city"]]

In [0]:
titles = pd.read_csv("/content/drive/My Drive/Università/3 ANNO MAGISTRALE/TESI/2_terms/scifi_metadata_htids.csv")[["htid", "title", "date"]]

In [0]:
titles["decade"] = (titles["date"]//10)*10

In [0]:
citytopic_books = pd.merge(left=titles, right=citytopic_htids, on="htid").sort_values("t36 city", ascending=False)

In [0]:
citytopic_books[:20]

,htid,title,date,decade,t36 city
19,hvd.32044021192513,The chessmen of Mars,1922,1920,0.075039
180,mdp.39015056811675,Inverted World,1974,1970,0.068370
15,uc2.ark:/13960/t2794143s,A princess of Mars,1917,1910,0.065164
61,mdp.39015056438271,The shadow girl,1946,1940,0.061538
186,mdp.39015062115814,Ragtime,1975,1970,0.057273
35,uc1.b3184615,Carson of Venus,1938,1930,0.055099
167,mdp.39015008587365,The Iron Dream,1972,1970,0.052903
9,dul1.ark:/13960/t1qf9cr59,A time of terror,1906,1900,0.051703
125,mdp.39015016450234,The Squares of the City,1965,1960,0.048525
317,mdp.39015054285773,The Years of Rice and Salt,2002,2000,0.040522


In [0]:
citytopic_books[:20].groupby("decade").count().htid

decade
1900    3
1910    2
1920    1
1930    2
1940    2
1960    2
1970    4
1980    1
2000    3
Name: htid, dtype: int64

In [0]:
citytopic_books.to_csv("/content/drive/My Drive/Università/3 ANNO MAGISTRALE/TESI/3_topicmodeling/citytopic_books.csv")